In [1]:
from selenium import webdriver
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
# del sys.path[0:0] # risky deletion

In [24]:
service = Service("/usr/lib/chromium-browser/chromedriver")

options = Options()
options.add_argument("--headless")
options.add_argument("--no-sandbox")
options.add_argument('--disable-dev-shm-usage')
driver = webdriver.Chrome(service=service, options=options)

### Visit Linkedin

In [26]:
driver.get("https://www.linkedin.com/")
print(driver.title)
from PIL import Image

driver.save_screenshot("web.png")
screenshot = Image.open("web.png")
screenshot.show()

LinkedIn: Log In or Sign Up


### Hidden: Sign in to Linkedin

In [ ]:
username = input('Enter your LinkedIn username: ')
password = input ('Enter your LinkedIn password: ')
username_field = driver.find_element(By.ID, 'session_key')
username_field.clear()
username_field.send_keys(username)
password_field = driver.find_element(By.ID, 'session_password')
password_field.clear()
password_field.send_keys(password)

In [28]:
submit_button = driver.find_element(By.CLASS_NAME, 'sign-in-form__submit-btn--full-width')
print(submit_button.text)
submit_button.click()

Sign in


In [29]:
print(driver.title)
driver.save_screenshot("web.png")
screenshot = Image.open("web.png")
screenshot.show()

Feed | LinkedIn


### Define function that scrolls to bottom of the web page to ensure dynamic loading of DOM elements and then extracts raw HTML

In [32]:
def get_jobs_data_from_linkedin(job_search_url):
    driver.get(job_search_url)
    time.sleep(5)
    print("web page title:", driver.title)              
    number_of_jobs = 25
    number_of_loops = 0
    loop_patience = 25
    raw_jobs_data = []
    while len(raw_jobs_data) < number_of_jobs:
        number_of_loops += 1
        raw_jobs_data = driver.execute_script("""
            let jobs_list = document.querySelectorAll('.job-card-container')
            jobs_list[jobs_list.length-1].scrollIntoView()
            return jobs_list
        """)
        time.sleep(3)
        raw_jobs_data = driver.execute_script("""
            let jobs_list = document.querySelectorAll('.job-card-container')
            jobs_list[jobs_list.length-1].scrollIntoView()
            return jobs_list
        """)
        if number_of_loops >= loop_patience:
            break
    print("Number of jobs found:", len(raw_jobs_data))
    return raw_jobs_data

### Define function for extracting text content from raw HTML tags

In [33]:
def extract_jobs_on_page(raw_jobs_data, jobs):
    for i in range(len(raw_jobs_data)):
        job = {}
        item = raw_jobs_data[i]
        try:
            company = item.find_element(By.CSS_SELECTOR, '.job-card-container__company-name').text
        except:
            company = ""
        try:
            job_title = item.find_element(By.CSS_SELECTOR, '.job-card-list__title').text
        except:
            job_title = ""
        try:
            location = item.find_element(By.CSS_SELECTOR, '.job-card-container__metadata-item').text
        except:
            location = ""
        try:
            posted = item.find_element(By.CSS_SELECTOR, '.job-card-container__listed-time').text
        except:
            posted = ""
        try:
            url = item.find_element(By.CSS_SELECTOR, '.job-card-container__link').get_attribute("href")
        except:
            url = ""

        job = {
            "company": company,
            "job_title": job_title,
            "location": location,
            "posted": posted,
            "url": url
        }
        jobs.append(job)
        if (i+1) % 5 == 0:
            print("Number of jobs extracted: ", (i + 1))

### Define function for extracting the necessary parts in the job description, namely the "description" and "skills" sections in the job listing

In [35]:
index_of_missing_descriptions = []

def extract_description_and_skills_from_jobs(jobs, full_jobs):
    number_of_missing_descriptions = 0
    for i in range(len(jobs)):  
        job = jobs[i]
        if job.get("url"):
            driver.get(job['url'])
            time.sleep(3)

#             description = driver.find_element(By.CSS_SELECTOR, '.jobs-description-content__text')
#             job['description'] = description.text
            times_number_subelements_exceed_two = 0
            description_elements = driver.execute_script("""
                let counter = 0
                let full_description = ""
                let missing_description = 0
                let description_div = document.querySelector('.jobs-description-content__text')
                if (!description_div){
                    missing_description += 1
                } else if (description_div.children && description_div.children.length > 2){
                    counter += 1
                } else if (description_div.children) {
                    full_description = description_div.children[1].innerHTML
                }
                return [counter, full_description, missing_description]
            """)
            times_number_subelements_exceed_two += description_elements[0]
            job['description'] = description_elements[1]
            if times_number_subelements_exceed_two > 0:
                print("div contains more than two children HTML tags:", times_number_subelements_exceed_two)
            
            number_of_missing_descriptions += description_elements[2]
            if (description_elements[2] > 0):
                index_of_missing_description.append(i)
                
            show_all_skills_button = driver.execute_script("""
                let show_all_skills_btn = null
                let artdeco_btns = document.querySelectorAll('.artdeco-button__text')
                for (const btn of artdeco_btns){
                  if (btn.textContent.includes('Show all skills')){
                    show_all_skills_btn = btn
                  }
                }
                return show_all_skills_btn
            """)

            if show_all_skills_button:
                show_all_skills_button.click()
                time.sleep(0.25)
                skills = []
                skills_list = driver.find_elements(By.CSS_SELECTOR, '.job-details-skill-match-status-list__unmatched-skill')
                for _skill in skills_list:
                    skills.append(_skill.text.split('\n')[0])
                job['skills'] = skills
            else:
                job['skills'] = []

            if len(description_elements[1]) > 0:
                full_jobs.append(job)
        
        if (i+1) % 10 == 0:
            print("Number of jobs fully extracted: ", (i + 1))
    
    print("Number of missing descriptions: ", number_of_missing_descriptions)
    return full_jobs, index_of_missing_descriptions

### Run the loop that calls previously defined helped functions and pushes the data into a Python list

In [17]:
search_url = 'https://www.linkedin.com/jobs/search/?currentJobId=3307342905&distance=25&f_E=1%2C2%2C3&f_T=25190%2C2732%2C25206&geoId=103644278&keywords=machine%20learning%20data%20scientist'

jobs = []

raw_jobs_data_page1 = get_jobs_data_from_linkedin(search_url)
extract_jobs_on_page(raw_jobs_data_page1, jobs)

for i in range(1, 40):
    print("Getting jobs from page: ", i+1)
    _search_url = search_url + '&start=' + str(i*25)
    raw_jobs_data = get_jobs_data_from_linkedin(_search_url)
    extract_jobs_on_page(raw_jobs_data, jobs)

print("All jobs extracted.")

web page title: machine learning data scientist Jobs | LinkedIn
Number of jobs found: 25
Number of jobs extracted:  5
Number of jobs extracted:  10
Number of jobs extracted:  15
Number of jobs extracted:  20
Number of jobs extracted:  25
Getting jobs from page:  2
web page title: machine learning data scientist Jobs | LinkedIn
Number of jobs found: 25
Number of jobs extracted:  5
Number of jobs extracted:  10
Number of jobs extracted:  15
Number of jobs extracted:  20
Number of jobs extracted:  25
Getting jobs from page:  3
web page title: machine learning data scientist Jobs | LinkedIn
Number of jobs found: 25
Number of jobs extracted:  5
Number of jobs extracted:  10
Number of jobs extracted:  15
Number of jobs extracted:  20
Number of jobs extracted:  25
Getting jobs from page:  4
web page title: machine learning data scientist Jobs | LinkedIn
Number of jobs found: 25
Number of jobs extracted:  5
Number of jobs extracted:  10
Number of jobs extracted:  15
Number of jobs extracted:  

Number of jobs found: 25
Number of jobs extracted:  5
Number of jobs extracted:  10
Number of jobs extracted:  15
Number of jobs extracted:  20
Number of jobs extracted:  25
Getting jobs from page:  33
web page title: machine learning data scientist Jobs | LinkedIn
Number of jobs found: 25
Number of jobs extracted:  5
Number of jobs extracted:  10
Number of jobs extracted:  15
Number of jobs extracted:  20
Number of jobs extracted:  25
Getting jobs from page:  34
web page title: machine learning data scientist Jobs | LinkedIn
Number of jobs found: 25
Number of jobs extracted:  5
Number of jobs extracted:  10
Number of jobs extracted:  15
Number of jobs extracted:  20
Number of jobs extracted:  25
Getting jobs from page:  35
web page title: machine learning data scientist Jobs | LinkedIn
Number of jobs found: 25
Number of jobs extracted:  5
Number of jobs extracted:  10
Number of jobs extracted:  15
Number of jobs extracted:  20
Number of jobs extracted:  25
Getting jobs from page:  36


In [39]:
fully_extracted_jobs = []

In [ ]:
# extract_description_and_skills_from_jobs(jobs[0:100], fully_extracted_jobs)

### Save the extracted data in a csv file

In [50]:
import csv

with open("linkedin_jobs_data.csv", "w", newline="", encoding="utf-8") as f:
    writer = csv.DictWriter(f, fieldnames=fully_extracted_jobs[0].keys())
    writer.writeheader()
    for job in fully_extracted_jobs:
        writer.writerow(job)

In [ ]:
# fully_extracted_jobs[0]